### Documentation
This notebook demonstrates how to remove remaining cosmic-rays, artifacts from the drizzled images. Various tools and steps are implemented to reduce the contamination. Sextractor is one of the tool that builds large catalog of sources, helps to identify cosmic-rays, bright objects and build photometry.

* [SExtractor](https://www.astromatic.net/software/sextractor/)
* [Roy Gal's Note on SExtractor](https://home.ifa.hawaii.edu/users/rgal/science/sextractor_notes.html)
* [Zhtools](http://hea-www.harvard.edu/RD/zhtools/)
* [Pyraf](https://pyraf.readthedocs.io/en/latest/)
* [Astroscrappy](https://pypi.org/project/astroscrappy/)

#### Package that I used for measuring flux, distance, etc  
[Astroptical](https://github.com/wwaldron/AstrOptical) by Will Waldron

### Step 1
After astrometry
- create a ds9 region covering chip gaps & edges #only support ellipse 
- save in the (FORMAT:ds9, COORD: image)
- Install SExtractor

``SExtractor is run from the shell with the following syntax:``
 
```bash

sex Image1 [Image2] -c configuration-file [-Parameter1 Value1 -Parameter2 Value2 ...]

#!/bin/bash

IMG_DIR=$HOME/ESO_137_002/OriginalFiles/08_drc_cosmicray
CONF_DIR=Photometry/OrigImages

# Run for F275W
sex $IMG_DIR/F275W_drc_chip_cleaned.fits[1] -c $CONF_DIR/ESO_F275W.conf

# Run for F475W
sex $IMG_DIR/F475W_drc_chip_cleaned.fits[1] -c $CONF_DIR/ESO_F475W.conf

# Run for F814W
sex $IMG_DIR/F814W_drc_chip_cleaned.fits[1] -c $CONF_DIR/ESO_F814W.conf

```

### Step 2 

- run finduncommonsrcs.ipyn to get region files for F275W, F475W, F814W. 

```python
from astroptical.sextractor import finduncommonsrcs
catFiles=['catfile_F275W.cat', 'catfile_F475W.cat', 'catfile_F814W.cat']
finduncommonsrcs(catFiles, roiFile='chip_gap_reg', maxPixSep=3, writeRegions=True,outDir=None, circRegRadius=5)#goodpixel=0 and badpixel=1
```

**Note**
The cat files are the catalog files from the SExtractor.The code will output 3 region files with uncommon sources in each filter. tweak circRegRadious to increase or decrease the size of region.

### Step 3

#### run Zhtools-mkreg command to create mask images
```bash
    mkreg refimg=F475W-ES0_137_002_drc.fits[1] out=F475W_cr_mask.fits reg=file:ESO_Orig_F475W_uncommon.reg    
 ```
**Note** remove 1st line/multiple lines from the reg files if you use mkreg 
- visually reinspect reg files 

### Step 4
**run PYRAF to create drc_chip.fits**
```bash
 
pyraf/ecl  

imcopy F814W_cr_mask.fits F814W_cr_mask.pl # Create copies of the masks

noao # Open the CRFIX Tool

imred

crutil 

crfix #mask undesired srcs 
 
Input image: F814W_drc.fits[1]

Output image: F814W_drc_chip.fits    

Cosmic ray mask: F814W_cr_mask.pl
```

### Step 4
- add header file at the end
```python
from astroptical.fitsutil import fixirafcrfix
 
fixirafcrfix('original_file', 'crfix')
```

### Step 5
(author: Will Waldron)
- using astroscrappy 
```python
from astroptical.cosmicray import createmask
#createmask(fileName, pssl=#, exposure=# ) 
createmask('F814W-IC_3418_drc.fits',exposure=1356,pssl=231)
```

**Note**

    pssl= SKY(background noise), add all background noise from *flc.fits
    e.g. for F475W, there are 2 FOV but altogether 4 SCI ext which need 
    to be added in the units of #/square arcsec. Alternatively, you can 
    add all 'MDRIZSKY' values from astrodrizzle.log
 
 
    exposure=EXPTIME, add all exp time #exposure in astrodrizzle.log or 
    header file of FITS, higher the pssl higher the detection, higher 
    the exposure lower the detection

### Step 6
When creating mask image, weight image is deleted from the FITS extension  
So don't forget to add weight image from the original drc image to create RMS image

In [16]:
drizzle_img='F814W-IC_3418_drc.fits'
clean_img='F814W-IC_3418_drc_chip_cleaned.fits'
with fits.open(drizzle) as fid:
    primHdr=fits.PrimaryHDU(header=fid[0].header)
    Wht=fits.ImageHDU(fid['WHT'].data, header=fid['WHT'].header)

with fits.open(clean) as hid:
    SciHdr=fits.ImageHDU(hid['SCI'].data, header=hid['SCI'].header)
    
        
hduListOut=fits.HDUList([primHdr,SciHdr,Wht])
hduListOut.writeto(clean,overwrite=True)


In [17]:
#check for the weight image 
with fits.open('../03_chip_gap/F814W-IC_3418_drc_chip_cleaned.fits') as fid:
    fid.info()

Filename: ../03_chip_gap/F814W-IC_3418_drc_chip_cleaned.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     847   ()      
  1  SCI           1 ImageHDU       426   (12025, 12114)   float32   
  2  WHT           1 ImageHDU        47   (12025, 12114)   float32   


### Create RMS images

```python
from astroptical.fitsutil import creatermsimage
creatermsimage('../03_chip_gap/F275W-IC_3418_drc_chip_cleaned.fits');
```